<a href="https://www.kaggle.com/code/mesutssmn/the-simpsons-character-prediction-cnn?scriptVersionId=167043337" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
train_dir = "/kaggle/input/the-simpsons-characters-dataset/simpsons_dataset"
test_dir = "/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset"

In [ ]:
train_img = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
test_img = [os.path.join(test_dir, f) for f in os.listdir(test_dir)]

In [ ]:
df_train = pd.read_csv("/kaggle/input/the-simpsons-characters-dataset/number_pic_char.csv")

In [ ]:
len(df_train.name.unique())

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=25,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(80,80),
                                                    batch_size=150,
                                                    subset = 'training',
                                                    class_mode='categorical',
                                                    color_mode='rgb')


In [ ]:
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split = 0.2)

In [ ]:
val_generator = val_datagen.flow_from_directory(train_dir,
                                                target_size=(80,80),
                                                batch_size=150,
                                                subset = 'validation',
                                                class_mode='categorical',
                                                color_mode='rgb')

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(80,80),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    color_mode='rgb')

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(80, 80, 3), activation="relu"))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation="relu")) 
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [ ]:
"""
conv_model = tf.keras.applications.VGG16(
                  weights='imagenet',
                  include_top=False,
                  input_shape=(80, 80, 3))
                  
model = tf.keras.models.Sequential()

model.add(conv_model)

model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(43, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=['acc'])

model.summary()"""

In [ ]:
callbacks = [
tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
    min_delta = 0.001,
    mode="max",
    patience = 10,
    verbose = 1,
    restore_best_weights = True),
    
tf.keras.callbacks.ModelCheckpoint(
    filepath='./model.keras',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)]

In [ ]:
history = model.fit(
    train_generator,
    epochs=30,
    callbacks=callbacks,
    validation_data=val_generator,
    batch_size=300)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Tranning_acc", color = "red")
plt.plot(epochs, val_acc, "b", label="validation_acc", color = "blue")
plt.legend()
plt.show()

In [ ]:
score = model.evaluate(val_generator,batch_size=32)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
saved_model = tf.keras.models.load_model('./model.keras')
saved_model

In [ ]:
img_path = "/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_28.jpg"
img = image.load_img(img_path, target_size=(64, 64, 3))
plt.imshow(img)
plt.show()